In [13]:
import pyaudio
import wave
import pandas as pd
import librosa
import glob 
import librosa.display
import os
import numpy as np
import tensorflow as tf
from threading import Thread
import time
from collections import deque
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle 


#About 10240 Chuck al secondo 
class Mic(Thread):
    def __init__(self, Format, Chunk, Channels, Rate, Sec, FileName, Path):
        ''' Constructor. '''
        
        Thread.__init__(self)
        self.Path = Path
        self.p = pyaudio.PyAudio()
        chuck_per_second = 20480
        self.Channels = Channels
        self.Format = Format
        self.Rate = Rate
        self.Running = True
        self.FileName = FileName
        self.Chunk = Chunk
        self.stream = self.p.open(format=Format,
                        channels=Channels,
                        rate=Rate,
                        input=True,
                        frames_per_buffer=Chunk)
        self.queue = deque(maxlen=(round(chuck_per_second*15/Chunk)))
 
    def run(self):
        try:
            print("Start acquisition")
            while self.Running:
                data = self.stream.read(Chunk, exception_on_overflow = False)
                self.queue.append(data)
        except:
            print("run() Error")
        
            
    def getAudioFile(self,fileName):
        try:   
            
            wf = wave.open(self.Path+fileName, 'wb')
            wf.setnchannels(self.Channels)
            wf.setsampwidth(self.p.get_sample_size(self.Format))
            wf.setframerate(self.Rate)
            wf.writeframes(b''.join(self.queue))
            wf.close()
        except:
            print("getAudioFile() Error")
            return ""
        
        
    def stop(self):
        self.Running = False

In [21]:
Chunk = 1024
Format = pyaudio.paInt16
Channels = 1
Rate = 22050
Sec = 15
Path = "DataSet.nosync/train-other-500/1/13/"
FileName = "test.flac"

#lb = LabelEncoder()
f_encoder = open('LabelEncoder.pkl', 'rb') 
lb = pickle.load(f_encoder)

f_scaler = open('StandardScaler.pkl', 'rb') 
ss = pickle.load(f_scaler)

model = tf.keras.models.load_model('saved_model/my_model')

mic = Mic(Format, Chunk, Channels, Rate, Sec, FileName, Path)
mic.start()

for i in range(71,81):
    time.sleep(15)
    print(i)
    mic.getAudioFile('1-13-00'+str(i)+".flac")
mic.stop()

Start acquisition
71
72
73
74
75
76
77
78
79
80
